In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

In [20]:
file = 'data/spb-rentals-2020-feb'
df = pd.read_csv(file + '.csv', encoding='utf-16', delimiter='\t')
df.head()

,ID,Дата,Тип жилья,Тип дома,Год постройки,Этаж,Адрес,X,Y,Станция метро,"Расстояние до метро, м","Общая площадь, м","Площадь кухни, м","Жилая площадь, м","Аренда, руб./мес.","Ком. платежи, руб./мес.","Комиссия, %","Комиссия, руб.","Общий платеж, руб./мес",Описание
0,373307,04.02.2021,студия,Кирпичный,1975 г.п.,6/16 этаж,"Наличная ул., 51",59.951063,30.234309,Приморская,280,20.0,NaN,NaN,18000,2500.0,50,9000.0,20500,"\r\n Просторна, тёплая квартира..."
1,373306,04.02.2021,студия,Кирпичный,2011 г.п.,16/17 этаж,"Ленинский пр., 84к2",59.858804,30.199274,Ленинский пр.,4010,30.0,2.0,NaN,19000,2000.0,50,9500.0,21000,\r\n Отличная квартира-студия н...
2,373288,04.02.2021,студия,Кирпичный,,2/6 этаж,"Подольская ул., 44",59.913083,30.328014,Технологический и-т,620,16.0,NaN,NaN,20000,3000.0,15,3000.0,23000,\r\n Сдам на длительный срок кв...
3,373273,04.02.2021,студия,Кирп.Монолит,,13/24 этаж,"Пулковская ул., 6к2",59.839669,30.350828,Звездная,740,25.0,NaN,NaN,18000,2500.0,60,10800.0,20500,\r\n Сдается квартира-студия в ...
4,373268,04.02.2021,студия,Монолит,,11/14 этаж,"Ковалевская ул., 26к1",59.979444,30.510961,Ладожская,6580,26.0,6.0,NaN,16000,3500.0,60,9600.0,19500,\r\n Сдам квартиру студию в хор...


In [21]:
grouped = df['Аренда, руб./мес.'].groupby(df['Станция метро']).mean()

In [14]:
new_df = pd.DataFrame(grouped)
new_df = new_df.sort_values(by='Аренда, руб./мес.', ascending=False)
new_df.head(20)

,"Аренда, руб./мес."
Станция метро,
Крестовский остров,253000.000000
Адмиралтейская,100750.000000
Петроградская,93708.333333
Чернышевская,92238.095238
Владимирская,88125.000000
Гостиный Двор,81266.666667
Спортивная,72727.181818
Невский пр.,70958.333333
Звенигородская,70428.571429


In [25]:
def group_by_station(flat_type):
    grouped = df[df['Тип жилья'] == flat_type][['Станция метро', 'Аренда, руб./мес.']]
    grouped = grouped['Аренда, руб./мес.'].groupby(grouped['Станция метро']).mean()
    grouped = pd.DataFrame(grouped)
    grouped = grouped.sort_values(by='Станция метро')
    grouped['Аренда, руб./мес.'] = grouped['Аренда, руб./мес.'].astype(int) 
    grouped = grouped.rename({'Аренда, руб./мес.': flat_type}, axis='columns')
    return grouped

In [27]:
group_by_station('2-комн. квартира')

,2-комн. квартира
Станция метро,
А.Hевского пл.,55666
Автово,28500
Адмиралтейская,72500
Академическая,24200
Балтийская,33333
...,...
Фрунзенская,43111
Черная речка,28142
Чернышевская,82384


In [29]:
df_m = pd.concat([group_by_station('студия'), group_by_station('1-комн. квартира'), 
                  group_by_station('2-комн. квартира'), group_by_station('3-комн. квартира')], 
                 axis=1, ignore_index=False, sort=False, join='outer')
df_m

,студия,1-комн. квартира,2-комн. квартира,3-комн. квартира
А.Hевского пл.,30333.0,48000.0,55666.0,85000.0
Автово,19333.0,18617.0,28500.0,41666.0
Адмиралтейская,55000.0,60000.0,72500.0,100000.0
Академическая,35500.0,20263.0,24200.0,26200.0
Беговая,21816.0,24629.0,42266.0,75869.0
...,...,...,...,...
Купчино,NaN,19416.0,24166.0,33750.0
Проспект Славы,NaN,22409.0,27545.0,35000.0
Удельная,NaN,23033.0,35272.0,63000.0
Электросила,NaN,33181.0,35000.0,53000.0


In [30]:
filename = 'data/rent-by-metro'
df_m.to_csv(filename + '.csv', encoding='utf-16')

In [33]:
file = 'data/rent-by-metro'
df3 = pd.read_csv(file + '.csv', encoding='utf-16', delimiter=',')
df3

,Unnamed: 0,студия,1-комн. квартира,2-комн. квартира,3-комн. квартира
0,А.Hевского пл.,30333.0,48000.0,55666.0,85000.0
1,Автово,19333.0,18617.0,28500.0,41666.0
2,Адмиралтейская,55000.0,60000.0,72500.0,100000.0
3,Академическая,35500.0,20263.0,24200.0,26200.0
4,Беговая,21816.0,24629.0,42266.0,75869.0
...,...,...,...,...,...
59,Купчино,NaN,19416.0,24166.0,33750.0
60,Проспект Славы,NaN,22409.0,27545.0,35000.0
61,Удельная,NaN,23033.0,35272.0,63000.0
62,Электросила,NaN,33181.0,35000.0,53000.0
